In [1]:
import argparse
import time
import numpy as np

import brainflow
from brainflow.board_shim import BoardShim, BrainFlowInputParams
from brainflow.data_filter import DataFilter, FilterTypes, AggOperations

import dynarray
   
params = BrainFlowInputParams() # default parameters, I suppose?
params.serial_port = '/dev/ttyUSB0' # board DEV

BoardShim.enable_board_logger() # so it's like that... shoddy, no?

In [2]:
%matplotlib qt
import matplotlib.pyplot as plt

In [3]:
board = BoardShim(2, params)
board.prepare_session()

In [4]:
#ringbuffer = 250 samples, it's not the fs
board.start_stream(250, '')

In [5]:
board.enable_dev_board_logger()

In [6]:
# board.config_board('/2') # I need to do this to en-able analogue mode !?

In [7]:
import pylsl

# F7 is secretly our Photo Diode!
ch_names = ['Fp1', 'Fp2', 'C3', 'C4', 'T5', 'T6', 'O1', 'O2', 
            'F7', 'F8', 'F3', 'F4', 'T3', 'T4', 'P3', 'P4', 'DIODE'];

sfreq = 125;

info = pylsl.StreamInfo('BrainFlow_eeg_run', 'EEG', len(ch_names), sfreq, 'float32', 'this_is_eeg_')

chns = info.desc().append_child("channels")
for label in ch_names:
    ch = chns.append_child("channel")
    ch.append_child_value("label", label)
    ch.append_child_value("unit", "microvolts")
    ch.append_child_value("type", "EEG")

outlet = pylsl.StreamOutlet(info)

In [ ]:
begin=time.time()

times=[]
shapes=[]
packet_ids=[]
timestamps=[]
packets=[]

d = dynarray.DynamicArray((None, 17))

import time
times_called=0
times_processed=0
sizes = []
while True: #time.time() < begin + 60*10:
    
    if board.get_board_data_count():
        times_called += 1

        m=board.get_board_data()
        
        eeg_data = m[[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16],:];
        # diode_data = m[[27, 28, 29],:];
        diode_data = m[[28],:];
        
        to_send = np.vstack((eeg_data, diode_data))
        # d.extend(to_send)
        
        # to_send = eeg_data
        
        sizes.append(to_send.T.shape)
        
        # d.extend(to_send.T)

        # now; put that eeg_data INTO a LSL stream, so we can check it
        # ALSO -- put the diode_data as well; but mark is as 'bad channel'.
        
        # this might impart maybe an extra ~1 msec on our time estimates:
        
        # outlet.push_chunk(all_data[b:e,:].tolist(), stamp)
        outlet.push_chunk(to_send.T.tolist())
        


In [10]:
to_send.shape

(16, 249)

In [10]:
m[1:17,:]

array([[ -23912.34179688],
       [ -97452.484375  ],
       [ -23237.09570312],
       [-102033.9921875 ],
       [-105942.7734375 ],
       [ -50615.0546875 ],
       [ -86593.9609375 ],
       [ -52005.48828125],
       [ -64316.47265625],
       [ -60398.25390625],
       [ -31083.1171875 ],
       [ -48795.66796875],
       [ -39251.359375  ],
       [-115229.515625  ],
       [ -61103.05078125],
       [ -66398.7578125 ]])

In [29]:
m

array([[ 9.10000000e+01],
       [-2.36997109e+04],
       [-9.17267734e+04],
       [-2.34906543e+04],
       [-1.07472656e+05],
       [-1.13700641e+05],
       [-5.06737500e+04],
       [-9.19576016e+04],
       [-5.14312734e+04],
       [-7.03715156e+04],
       [-6.10497188e+04],
       [-3.23168438e+04],
       [-4.86502461e+04],
       [-4.01684297e+04],
       [-1.13348352e+05],
       [-6.42608164e+04],
       [-6.51007930e+04],
       [ 0.00000000e+00],
       [ 0.00000000e+00],
       [ 0.00000000e+00],
       [ 1.93000000e+02],
       [ 1.00000000e+00],
       [ 1.45500000e+02],
       [ 3.00000000e+00],
       [ 2.51000000e+02],
       [ 3.00000000e+00],
       [ 2.51500000e+02],
       [ 4.01500000e+02],
       [ 1.01900000e+03],
       [ 1.01950000e+03],
       [ 1.60076275e+09]])

In [7]:
times_called_fit()

In [80]:
board.stop_stream()

In [81]:
board.release_session()

# This should have done it

0

In [81]:
plt.figure()
plt.plot(m[22,:] - m[22,0])

In [44]:
m.shape

(31, 124)

In [53]:
plt.figure(); plt.plot(m[[2, 3],:].T)

In [101]:
board.stop_stream()

In [102]:
board.release_session()

In [77]:
board.get_other_channels(2)

[20, 21, 22, 23, 24, 25, 26]

In [66]:
import matplotlib.pyplot as plt
%matplotlib qt
plt.figure();plt.imshow(d[0:16])

In [75]:
plt.figure();plt.plot(np.diff(time.time() - d[30]))

In [49]:
board.stop_stream()

In [4]:
time.sleep (10)
# data = board.get_current_board_data (256) # get latest 256 packages or less, doesnt remove them from internal buffer

In [4]:
while True:
    data = board.get_board_data () # get all data and remove it from internal buffer
    
    
board.stop_stream()
board.release_session()

    print (data)

In [18]:
default_16_ch_names = ['Fp1',  'Fp2',  'C3',  'C4',  'T5',  'T6',  'O1',  'O2',  'F7',  'F8',  'F3',  'F4',  'T3',  'T4',  'P3',  'P4']
default_8_ch_names = ['Fp1', 'Fp2', 'C3', 'C4', 'T5', 'T6', 'O1', 'O2']


default_data_encoding = 'float32'

default_eeg_stream_name = 'openbci_eeg'
default_aux_stream_name = 'openbci_aux'
random_number = random.randint(0, 999)
default_eeg_stream_identifier = 'openbci_eeg_id{:3d}'.format(random_number)
default_aux_stream_identifier = 'openbci_aux_id{:3d}'.format(random_number)

if board.get_sampling_rate(2) == 125:
    ch_names = default_16_ch_names
elif board.get_sampling_rate(2) == 250:
    ch_names = default_16_ch_names

other_channels = ['X','Y','Z']

In [ ]:
# make the info for the EEG:
info_eeg = pylsl.StreamInfo(default_eeg_stream_name, 'EEG', len(ch_names), board.get_sampling_rate(2), 'float32', default_eeg_stream_identifier)
chns = info_eeg.desc().append_child("channels")
for label in ch_names:
    ch = chns.append_child("channel")
    ch.append_child_value("label", label)
    ch.append_child_value("unit", "microvolts")
    ch.append_child_value("type", "EEG")

# make the info for the AUX:
info_aux = pylsl.StreamInfo(default_aux_stream_name, 'AUX', len(ch_names), board.get_sampling_rate(2), 'float32', default_aux_stream_identifier)
chns = info_aux.desc().append_child("channels")
for label in other_channels:
    ch = chns.append_child("channel")
    ch.append_child_value("label", label)
    ch.append_child_value("unit", "microvolts")
    ch.append_child_value("type", "EEG")

    
outlet_eeg = pylsl.StreamOutlet(info_eeg)
outlet_aux = pylsl.StreamOutlet(info_aux)

In [2]:
import numpy as np

In [4]:
import math

In [6]:
import random

'335'

In [ ]:
# to make a LSL outlet, we need to set all kinds of parameters.

    info = pylsl.StreamInfo('Playback-3', 'EEG', len(raw.ch_names), raw.info['sfreq'], 'float32', 'someidentifier123')

    chns = info.desc().append_child("channels")
    for label in raw.ch_names:
        ch = chns.append_child("channel")
        ch.append_child_value("label", label)
        ch.append_child_value("unit", "microvolts")
        ch.append_child_value("type", "EEG")

    outlet = pylsl.StreamOutlet(info)


def send_it(data_file):

    import mne
    # data_file = '/home/johan/nf/nf-rtime/Projects_Templates/3_RTMRICWL/trial_data_for_mri_cwl_development.set'

    raw = mne.io.read_raw_eeglab(data_file).resample(500)
    raw_scaling = [item['cal'] for item in raw.info['chs']]
    sfreq=raw.info['sfreq']
    
    print(raw)
    # create info for our purposes:
    info = pylsl.StreamInfo('Playback-3', 'EEG', len(raw.ch_names), raw.info['sfreq'], 'float32', 'someidentifier123')

    chns = info.desc().append_child("channels")
    for label in raw.ch_names:
        ch = chns.append_child("channel")
        ch.append_child_value("label", label)
        ch.append_child_value("unit", "microvolts")
        ch.append_child_value("type", "EEG")

    outlet = pylsl.StreamOutlet(info)

    # time_to_send_new_data_point = time.time()
    # time_to_wait_between_sending_data_points = 1/sfreq

    begin_time = time.time()
    current_time = 0.0
    all_data = raw.get_data().T / raw_scaling

    inds_sent = np.zeros(raw.last_samp+1, dtype=bool)
    while current_time < raw.last_samp/raw.info['sfreq']:

        inds_current = raw.times < current_time
        bool_slice = inds_current & ~inds_sent
        if bool_slice.any():
            b=np.min(np.where(bool_slice)[0])
            e=np.max(np.where(bool_slice)[0])+1


            stamp = pylsl.local_clock()-0.125
            outlet.push_chunk(all_data[b:e,:].tolist(), stamp)
            inds_sent = inds_sent | inds_current

            no_data_time=0

        time.sleep(0.01)
        current_time = time.time() - begin_time

    print('We are done sending this data')
    del(outlet)
        